# NASA SPoRT SST vs. In-Situ Observations Jun 1. - Aug. 31
## Cayuga Lake, NY

For each Lake for June - September 2020 (or October 2020) period:

1)  Time series plot of SPoRTS temps vs. in-situ temp obs

2)  Scatterplot of  SPoRTS temps vs. in-situ temp obs

     a) plot best fit line
     b) plot 1:1 reference line
     c) indicate  correlation coefficient value

3) Bar Graph of Mean Algebraic Error (Y axis)  vs. Time of Day (x axis)  [Hourly 0, 1, 2, ....23 UTC] for entire period
    purpose - to see if there is a diurnal bias in the SPoRTS temps)

4) Skill Statistics

     a) Mean Algebraic  Error 
     b) Mean Absolute Error
     b) RMSE

5) Map depicting the lakes in NE USA with a symbol indicating which lakes were evaluated in the study.

### Imports

In [ ]:
import os, sys
import ast
from datetime import date, datetime
from decimal import Decimal
from math import sqrt

from netCDF4 import Dataset
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, shape
import matplotlib.pyplot as plt
import contextily as ctx
from pyproj import Transformer
import plotly.express as px
import plotly.graph_objects as go
import django
from sklearn.metrics import mean_squared_error, mean_absolute_error
import geopy.distance

In [ ]:
# setup for django
sys.path.append(os.path.abspath(os.path.join('..', 'djangoapp')))
os.environ["DJANGO_SETTINGS_MODULE"] = "config.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from lib import utils
from api import models

## Get Data

In [ ]:
# get lake model object
lake = models.Lake.objects.filter(name__contains='Cayuga').first()

In [ ]:
# load lake geometry from saved geojson
geom = [ shape(eval(lake.geojson)['features'][0]['geometry']) ]

lake_geom = gpd.GeoDataFrame({'geometry': geom})
lake_geom.crs = "EPSG:4326"
lake_geom = lake_geom.to_crs(epsg=3857)

### Lake Shape Plot

In [ ]:
# plot lake boundary
fig= plt.figure(figsize=(10, 10))

ax = plt.plot(*lake_geom['geometry'][0].exterior.xy, color='red', linestyle='dashed', linewidth=2)
ax = plt.gca()

ax.set_xlim([ float(lake_geom.bounds['minx'])-50000, float(lake_geom.bounds['maxx'])+50000 ])
ax.set_ylim([ float(lake_geom.bounds['miny'])-50000, float(lake_geom.bounds['maxy'])+50000 ])

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=9)
ax.set_axis_off()

### Get SPoRT SST Data

In [ ]:
sst_lake_points = utils.get_sst_output_for_lake(lake, date(2020, 6, 1), date(2020, 8, 31))

In [ ]:
# -3.76 is what would be a pixel value of 0 in the original tiff file
# seems to be the error value, makes the plots look terrible
sst_lake_points['water_temp'] = sst_lake_points['water_temp'].replace(Decimal('-3.76'), None)

### Get 5 SST Points Closest to In-Situ Station Location

In [ ]:
# get station
station = models.Station.objects.filter(lake=lake).first()

In [ ]:
dists = {}
for idx, group in sst_lake_points.groupby('grid_idx'):
    point = group['geometry'].unique()[0]
    dist = geopy.distance.geodesic((station.lon, station.lat), (point.x, point.y)).m
    dists[str(idx)] = dist

In [ ]:
closest_5_idx = sorted(dists, key=dists.__getitem__)[:5]
closest_5_idx

In [ ]:
closest_5 = sst_lake_points[sst_lake_points['grid_idx'].isin(closest_5_idx)]

### SST Points vs. Station Location Plot

In [ ]:
# plot lake boundary
fig = plt.figure(figsize=(10, 10))

transformer = Transformer.from_crs("epsg:4326", "epsg:3857")
for idx in sst_lake_points['grid_idx'].unique():
    lon, lat = list(sst_lake_points[sst_lake_points['grid_idx'] == idx][['lon', 'lat']].iloc[0])
    x, y = transformer.transform(lat, lon)
    if idx in closest_5_idx:
        plt.plot(x, y, 'ok', markersize=5, color='red', label='Closest 5 Points')
    else:
        plt.plot(x, y, 'ok', markersize=5, color='blue', label='Additional Lake Points')
        
station_x, station_y = transformer.transform(station.lat, station.lon)
plt.plot(station_x, station_y, 'ok', markersize=5, color='black', label='Station (In-Situ Obs)')

ax = plt.gca()

ax.set_xlim([ float(lake_geom.bounds['minx'])-50000, float(lake_geom.bounds['maxx'])+50000 ])
ax.set_ylim([ float(lake_geom.bounds['miny'])-50000, float(lake_geom.bounds['maxy'])+50000 ])

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=9)
ax.set_axis_off()

def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique))

legend_without_duplicate_labels(ax)

plt.show()

### Get In-Situ Data

In [ ]:
# load cayuga data from csv
insitu_data = pd.read_csv('../data/insitu/JuneJulyAugust2020_Cayuga_Lake_Water_Temperature.csv', skiprows=1, parse_dates=[0])
insitu_data.head()

## All SPoRTS SST Temps vs. In-Situ Temp Observations

In [ ]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

for idx in sst_lake_points['grid_idx'].unique():
    fig.add_trace(
        go.Scatter(
            x=sst_lake_points[sst_lake_points['grid_idx'] == idx]['datetime'],
            y=sst_lake_points[sst_lake_points['grid_idx'] == idx]['water_temp'],
            mode='lines',
            name=idx
        )
    )
    

fig.add_trace(
    go.Scatter(
        x=insitu_data['Timestamp'],
        y=insitu_data['Avg'],
        mode='lines',
        name="In-Situ",
        line={ 'color': 'black' }
    )
)

fig.update_layout(
    title="SST Lake Model Cycles 06 and 18 and In-Situ Temp for Cayuga Lake, NY from June 1, 2020 - Aug 31, 2020",
    xaxis_title="Forcast Cycle/Observation Datetime",
    yaxis_title="Water Temp (C)",
    xaxis = {
        'dtick': 3600000.0*24#*7
    },
    width=1920, 
    height=1080
)
fig.update_traces(connectgaps=False)

fig.show()

## Closest 5 SPoRTS SST Temps vs. In-Situ Temp Observations

In [ ]:
# Create traces
fig = go.Figure()

for idx in closest_5_idx:
    fig.add_trace(
        go.Scatter(
            x=sst_lake_points[sst_lake_points['grid_idx'] == idx]['datetime'],
            y=sst_lake_points[sst_lake_points['grid_idx'] == idx]['water_temp'],
            mode='lines',
            name=idx
        )
    )
    

fig.add_trace(
    go.Scatter(
        x=insitu_data['Timestamp'],
        y=insitu_data['Avg'],
        mode='lines',
        name="In-Situ",
        line={ 'color': 'black' }
    )
)

fig.update_layout(
    title="Closest 5 SST Lake Model Points (Cycles 06 and 18) and In-Situ Temp for Lake Cayuga, NY from June 1, 2020 - Aug 31, 2020",
    xaxis_title="Forcast Cycle/Observation Datetime",
    yaxis_title="Water Temp (C)",
    xaxis = {
        'dtick': 3600000.0*24#*7
    },
    width=1920, 
    height=1080
)
fig.update_traces(connectgaps=False)

fig.show()

## Skill Statistics with 5 Closest Points

In [ ]:
y_true_df = insitu_data[((insitu_data['Timestamp'].dt.hour == 6) & (insitu_data['Timestamp'].dt.minute == 0)) | ((insitu_data['Timestamp'].dt.hour == 18) & (insitu_data['Timestamp'].dt.minute == 0))].sort_values(by='Timestamp')
y_true_df.head()

In [ ]:
y_pred_df = sst_lake_points[sst_lake_points['grid_idx'].isin(closest_5_idx)].sort_values(by='datetime')
y_pred_df.head()

In [ ]:
assert(len(y_pred_df['datetime'].unique()) == len(y_true_df['Avg'].values))

In [ ]:
# convert 'water_temp' column to numeric type so it can be averaged over
y_pred_df['water_temp'] = pd.to_numeric(y_pred_df['water_temp'])# , errors ='ignore')

In [ ]:
# average closest 5 SST points for 0600 and 1800
y_pred = y_pred_df.groupby('datetime')['water_temp'].mean()

### Mean Absolute Error

In [ ]:
# MAE
mean_absolute_error(y_true_df['Avg'].values, y_pred.values)

### Root Mean Squared Error

In [ ]:
# RMSE
sqrt(mean_squared_error(y_true_df['Avg'].values, y_pred.values))

In [ ]:
# Create traces
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=y_true_df['Avg'].values,
        y=y_pred.values,
        mode='markers',
        name="In-Situ"
    )
)

fig.add_trace(
    go.Scatter(
        x=list(range(14, 30)),
        y=list(range(14, 30)),
        mode='lines',
        name="1:1"
    )
)

fig.update_layout(
    title="Observed vs Predicted Temp",
    xaxis_title="Observed Temp (C)",
    yaxis_title="Predicted Temp (C)",
)
fig.update_traces(connectgaps=False)

fig.show()

In [ ]:
# Create traces
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=y_true_df['Timestamp'],
        y=abs(y_pred.values - y_true_df['Avg'].values),
        name="Absolute Error",
    )
)

fig.add_trace(
    go.Scatter(
        x=insitu_data['Timestamp'],
        y=insitu_data['Avg'],
        mode='lines',
        name="In-Situ",
        line={ 'color': 'black' }
    )
)

fig.update_layout(
    title="Absolute Error",
    xaxis_title="Forcast Cycle/Observation Datetime",
    yaxis_title="Temp (C)",
)

fig.show()